In [1]:
%pip install -r requirements.txt

import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [148]:
df = pd.read_csv('./data/before-convert/interest_rate.csv')
df['year'] = pd.DatetimeIndex(df['TIME']).year
df['month'] = pd.DatetimeIndex(df['TIME']).month

df = df.drop('TIME', axis=1)

df = df.rename(columns = {'LOCATION':'location'})
df = df.drop('Country', axis=1)

longTerm = df.query('Subject == "Long-term interest rates, Per cent per annum"')
longTerm = longTerm.rename(columns = {'Interest-Rate':'interest_rate_long_term'})
longTerm = longTerm.drop('Subject', axis=1)

shortTerm = df.query('Subject == "Short-term interest rates, Per cent per annum"')
shortTerm = shortTerm.rename(columns = {'Interest-Rate':'interest_rate_short_term'})
shortTerm = shortTerm.drop('Subject', axis=1)


immediateTerm = df.query('Subject == "Immediate interest rates, Call Money, Interbank Rate, Per cent per annum"')
immediateTerm = immediateTerm.rename(columns = {'Interest-Rate':'interest_rate_immediate_term'})
immediateTerm = immediateTerm.drop('Subject', axis=1)

df = pd.merge(longTerm, shortTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = pd.merge(df, immediateTerm, on=['location', 'year', 'month'] , how='inner', suffixes=('', '_delme'))
df = df[[c for c in df.columns if not c.endswith('_delme')]]

df.to_csv('./data/interest_rate.csv', index=False)

In [158]:
df_au = pd.read_csv('./data/before-convert/cpi_au.csv')

df_au['year'] = pd.DatetimeIndex(df_au['TIME']).year
df_au['month'] = pd.DatetimeIndex(df_au['TIME']).month

df_au = df_au.drop('TIME', axis=1)

df_au.head()

df_us = pd.read_csv('./data/before-convert/cpi_us.csv')

df_final_us = pd.DataFrame(columns=['year','month'])


for index, row in df_us.iterrows():
    for i in range(12):
        df_final_us = pd.concat([df_final_us, pd.DataFrame.from_records([{'year':row[0],'month':i+1, 'cpi-index':row[i+1]}])])

df_final_us['location'] = 'US'
df_final_us = df_final_us.dropna()
df_final_us['year'] = df_final_us['year'].astype(int)
    
df = pd.concat([df_au, df_final_us])

df.to_csv('./data/cpi.csv', index=False)